In [67]:
import matplotlib.pyplot as plt
import numpy as np

from numpy import sin, cos, pi, arcsin, inf
%matplotlib inline

In [121]:
from scipy import integrate
x = np.linspace(0,10,2)
y = x
dx = abs(x[1]-x[0])
integrate.simps(y, x, axis=0)

50.0

In [126]:
def func(i):
    return i

In [127]:
x

array([  0.,  10.])

In [130]:
y = np.zeros((2, 5))
for i, blag in enumerate(x):
    y[i, :] += func(blag)

In [131]:
y

array([[  0.,   0.,   0.,   0.,   0.],
       [ 10.,  10.,  10.,  10.,  10.]])

In [133]:
np.shape(x)

(2,)

In [134]:
np.shape(y)

(2, 5)

In [137]:
integrate.simps(y, x, axis=0)

array([ 50.,  50.,  50.,  50.,  50.])

In [89]:
x

array([  0.,  10.])